# 天気をLine通知する方法
### Open Weather API　と　LineNotify を使用する

### 各モジュールインポート

In [ ]:
import datetime
import requests
import json
import pandas as pd
import os
import sys

from pytz import timezone

### LineNotifyのTOKEN（トークン）とAPI
### Open Weather APIのCity（都市）とAPIKEY（キー）とAPI

In [ ]:
#LineNotifyのトークンとAPI
line_token = "G22urEDafbd2guYIVNmnla2S5Mub1wP21UYAHq8xzFW"
line_notify_api = 'https://notify-api.line.me/api/notify'

#天気取得のためのOpen Weather APIのCity(都市）とAPIkeyとAPI
City = "Sabae,JP"
open_weather_api_key = "e1684e4b730562d2c3868bec6b1932a9"
open_weather_api = 'http://api.openweathermap.org/data/2.5/forecast?q={city}&units=metric&lang=ja&cnt=8&APPID={key}'

#forecast : 3時間ごとのデータを取得
#units : 気温の単位の指定　metric(摂氏℃になる)
#lang : 表記言語
#cnt : 3時間ごといくつとるか(8つ＝24時間)

### 天気アイコンの作成

In [ ]:
def getWeatherIcon(icon_str):
    if (
        icon_str == "01d" 
        or icon_str == "01n"
    ):
        return "☀"
    elif (
        icon_str == "02d"
        or icon_str == "02n"
        or icon_str == "03d"
        or icon_str == "03n"
        or icon_str == "04d"
        or icon_str == "04n"
    ) :
        return "☁"
    elif (
        icon_str == "09d"
        or icon_str == "09n"
        or icon_str == "10d"
        or icon_str == "10n"
    ):
        return "☂"
    elif (
        icon_str == "11d"
        or icon_str == "11n"
    ):
        return "⚡"
    elif (
        icon_str == "13d"
        or icon_str == ""
    ):
        return "⛄"
    else:
        return ""

### LineNotifyでLine通知する

In [ ]:
def lineNotifyWeather(df):
    texts = []
    for date, item in df:
        texts.append(f"\n【{date}】\n")
        for _, d in item.iterrows():
            texts.append(
                f"{d['time']}時 {getWeatherIcon(d['icon'])} {d['temp']}(℃) {d['rain']}(mm/3h)\n"
            )
    
    url = line_notify_api
    token = line_token
    headers = {'Authorization' : 'Bearer ' + token}
    message = texts
    payload = {'message': message}
    p = requests.post(url, headers=headers, data=payload)
    print (p)

### Open Weather APIから天気を取得して通知

In [ ]:
def getTheWeatherAndNotify():
    url = open_weather_api.format(city=City, key=open_weather_api_key)
    response = requests.get(url)
    forecastData = json.loads(response.text)
    
    data_rj = []
    for item in forecastData['list']:
        data_list = {}
        timestamp = timezone('Asia/Tokyo').localize(datetime.datetime.fromtimestamp(item['dt']))
        data_list['date'] = timestamp.strftime('%m/%d %a')
        data_list['time'] = timestamp.strftime('%H')
        data_list['description'] = item['weather'][0]['description']
        data_list['icon'] = item['weather'][0]['icon']
        data_list['temp'] = round(item['main']['temp'])
        data_list['rain'] = round(item['rain']['3h'], 1) if 'rain' in item else 0
        data_rj.append(data_list)
    mes = pd.DataFrame(data_rj).groupby('date')
    lineNotifyWeather(mes)


### 各クラスの実行

In [ ]:
getTheWeatherAndNotify()